### Group Members:  Jo Ann Elias, Mariko Falke  &  Marcia Cosgrove

# UCD Bootcamp Visualization Project 

Data Source: https://data.world/awram/us-mass-shootings


#  (E) - Data Extraction

* Import Needed Libraries
* Load Excel File
* Read Excel File & Store in Dataframe

In [1]:
#####  *** Set Up ***  #####

### Import Dependencies
import pandas as pd
import numpy as np
import datetime as dt

### Python SQL Toolkit & Object Relational Mapper (ORM)
import psycopg2
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

### Display Options - Set Dataframe to Show All Columns in Output Window
pd.set_option("display.max_columns", None)


In [2]:
### File to Load 
US_Shooting_data = "Resources/US_Mass_Shootings_Data_1982_2019.xlsx"

### Read Excel Files & Store in Pandas Dataframes
US_Shooting_df = pd.read_excel(US_Shooting_data, encoding='unicode_escape')

FileNotFoundError: [Errno 2] No such file or directory: 'Resources/US_Mass_Shootings_Data_1982_2019.xlsx'

In [3]:
#### Check Dataframe Datatypes
# US_Shooting.dtypes

#### Display Dataframe
US_Shooting_df.head()

NameError: name 'US_Shooting_df' is not defined

In [4]:
#### Display Statistical Summary
US_Shooting_df.describe()

NameError: name 'US_Shooting_df' is not defined

# (T) - Data Transformation

* Drop Columns that are Not Needed
* Create Columns that are Needed
* Reorder Columns 
* Recode Field Values for Consistency & Analysis


In [5]:
#### Drop Columns that Aren't Needed (we are not doing text analysis, so dropping notes fields/string fields)
US_Shooting_df = US_Shooting_df.drop(columns=["mental_health_details", "summary", "where_obtained", "weapon_details", 
                                      "mental_health_sources", "sources_additional_age", "type", "weapon_type",
                                       "prior_signs_mental_health_issues", "weapons_obtained_legally"], axis=1)

#### Create Additional Date Field
US_Shooting_df["month"] = US_Shooting_df["date"].dt.month


#### Rename One of the "Location"  Fields (dataset came with two location fields) 
US_Shooting_df = US_Shooting_df.rename(columns={"location": "city_state", 
                                          "location.1": "site"})

#### Reorder Dataframe
US_Shooting_df = US_Shooting_df[["case", "city_state", "site",
                           "date", "month", "year",
                           "injured", "fatalities","total_victims",
                          "age_of_shooter", "race",
                          "gender", "longitude", "latitude"]]

#### Display Dataframe
US_Shooting_df.head()

NameError: name 'US_Shooting_df' is not defined

In [6]:
#####  *** Recode Fields for Consistency & Analysis  ***  #####

### Recode "age_of_shooter"
US_Shooting_df["age_of_shooter"].replace({"-": 0}, inplace=True)

### Recode "race"
US_Shooting_df["race"].replace({"-": "Other/Unknown", 
                              "unclear": "Other/Unknown",
                              "Other":  "Other/Unknown",
                              "White ": "White",
                              "white": "White",
                               "black": "Black"}, inplace=True)


### Recode "gender"
US_Shooting_df["gender"].replace({"-": "Other/Unknown", 
                              "Male & Female": "Other/Unknown",
                              "F":  "Female",
                              "M": "Male"}, inplace=True)

### Recode "site"
## This isn't recoding correctly, go back and revisit
US_Shooting_df["site"].replace({"Other\n ": "Other", 
                              "\nWorkplace ": "Workplace"}, inplace=True)


NameError: name 'US_Shooting_df' is not defined

In [7]:
#### Display Dataframe
US_Shooting_df.head()

NameError: name 'US_Shooting_df' is not defined

# (L) - Data Load

* Create Engine & Connection Layer to Database
* Use Pandas to Load CSV Converted Dataframe to PostgreSQL DB
* Create a Session (link) from Python to PostgreSQL DB


## Prior to Executing the Following Code, Set Up PostgreSQL Database

* Using pgAdim, create a new database named "Mass_Shootings" (right-click on Databases in left menu list, create "database")
* Click on the new database and navigate under "Schemas" > "Public" > "Tables" and create a new table named "Mass_Shootings" (right-click, create "table")
* Navigate to "Columns", right-click and create "column". A new pop up window will display and you need to fill out the following elements:<br>

    1) "General Tab" > "Name" -  name as it appears in the list below<br>
    2) "Definition Tab" > "Data Type" - type is shown in list below<br>
    3) Repeat the addition of columns for entire list (14 columns total)


In [8]:
from IPython.display import Image
Image("Resources/Mass_Shootings_Columns.png")

FileNotFoundError: No such file or directory: 'Resources/Mass_Shootings_Columns.png'

FileNotFoundError: No such file or directory: 'Resources/Mass_Shootings_Columns.png'

<IPython.core.display.Image object>

In [9]:
##### ***  Create Engine & Connection Layer ***  #####
connection_string = "postgres:Skyriot12!@localhost:5432/Mass_Shootings"
engine = create_engine(f'postgresql://{connection_string}')
connection = engine.connect()

In [10]:
##### *** Use Pandas to Load CSV Converted Dataframe into Database *** #####
US_Shooting_df.to_sql(name="Mass_Shootings", con=engine, if_exists="replace", index=False)

NameError: name 'US_Shooting_df' is not defined

In [11]:
### Check Table Loaded to Database
engine.table_names()

['Mass_Shootings']

In [12]:
##### ***  Create Our Session (link) from Python to the Database ***  #####
session = Session(engine)

In [13]:
### Confirm data has been added by querying the database
pd.read_sql_table("Mass_Shootings", con=engine).head()

,case,city_state,site,date,month,year,injured,fatalities,total_victims,age_of_shooter,race,gender,longitude,latitude
0,Molson Coors shooting,"Milwaukee, Wisconsin",Workplace,2020-02-26,2,2020,0,5,5,51,Black,Male,-87.962537,43.044511
1,Jersey City kosher market shooting,"Jersey City, New Jersey",Other,2019-12-10,12,2019,3,4,7,0,Black,Other/Unknown,-74.083609,40.707363
2,Pensacola Naval base shooting,"Pensacola, Florida",Military,2019-12-06,12,2019,8,3,11,0,Other/Unknown,Male,-87.288567,30.364707
3,Odessa-Midland shooting spree,"Odessa, Texas",Other,2019-08-31,8,2019,25,7,32,36,White,Male,-102.279600,31.925974
4,Dayton entertainment district shooting,"Dayton, Ohio",Other,2019-08-04,8,2019,27,9,36,24,White,Male,-84.184947,39.757312


### NOTE:  In order to review the data in pgAdmin, you will need to right-click on the Mass_Shootings database in pgAdmin and select "refresh" the database.